In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [17]:
df = pd.read_csv('titanic/train.csv')
train = df.sample(frac=0.7)
test = df.drop(df.sample(frac=0.7).index)
# train_index = np.arange(df.shape[0])

In [ ]:
label = 'Survived'
save_path = 'agModels-predictClass'
predictor = TabularPredictor(label=label, path=save_path).fit(train,presets='best_quality')

In [ ]:
predictor.leaderboard(train, silent=True)

In [ ]:
y_test = test[label] 
test_data_nolab = test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
predictor.leaderboard(test, silent=True)

In [36]:
pre_data = pd.read_csv('titanic/test.csv')

In [37]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(pre_data)

In [39]:
result = pd.DataFrame(pre_data['PassengerId'])
result['Survived'] = y_pred

In [40]:
result.to_csv('titanic/submission.csv',index=False)